<a href="https://colab.research.google.com/github/CesarSarmiento1111/MetNumUN2024II/blob/main/LabATQ/LabATQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import logging

import xarray as xr  # xarray for data manipulation

import qnt.data as qndata     # functions for loading data
import qnt.backtester as qnbt # built-in backtester
import qnt.ta as qnta         # technical analysis library
import qnt.stats as qnstats   # statistical functions

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

np.seterr(divide = "ignore")

from qnt.ta.macd import macd
from qnt.ta.rsi  import rsi
from qnt.ta.stochastic import stochastic_k, stochastic, slow_stochastic

from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error

from sklearn.ensemble import HistGradientBoostingClassifier

In [ ]:
stock_data = qndata.stocks_load_spx_data(min_date='2005-06-01')

100% (367973 of 367973) |################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (79197 of 79197) |##################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (13022844 of 13022844) |############| Elapsed Time: 0:00:00 Time:  0:00:00


fetched chunk 1/13 1s


100% (13022848 of 13022848) |############| Elapsed Time: 0:00:00 Time:  0:00:00


fetched chunk 2/13 2s


100% (13022844 of 13022844) |############| Elapsed Time: 0:00:00 Time:  0:00:00


...

fetched chunk 11/13 10s


100% (13022736 of 13022736) |############| Elapsed Time: 0:00:00 Time:  0:00:00


fetched chunk 12/13 11s


100% (7631572 of 7631572) |##############| Elapsed Time: 0:00:00 Time:  0:00:00


fetched chunk 13/13 11s
Data loaded 12s


In [ ]:
def get_features(data):
    """Builds the features used for learning:
       * a trend indicator;
       * the moving average convergence divergence;
       * a volatility measure;
       * the stochastic oscillator;
       * the relative strength index;
       * the logarithm of the closing price;
       * Trix indicator.
       * Average True Range (ATR);
       * On-Balance Volume (OBV).
       These features can be modified and new ones can be added easily.
    """

    # trend:
    #trend = qnta.roc(qnta.lwma(data.sel(field="close"), 60), 1)

    # Liquidity filter (1.0 for liquid assets, 0.0 for non-liquid assets):
    liq = data.sel(field="is_liquid").ffill("time").bfill("time").fillna(0)

    # moving average convergence divergence (MACD):
    macd = qnta.macd(data.sel(field="close"))
    macd2_line, macd2_signal, macd2_hist = qnta.macd(data, 12, 26, 9)

    # volatility:
    volatility = qnta.tr(data.sel(field="high"), data.sel(field="low"), data.sel(field="close"))
    volatility = volatility / data.sel(field="close")
    volatility = qnta.lwma(volatility, 14)

    # the stochastic oscillator:
    #k, d = qnta.stochastic(data.sel(field="high"), data.sel(field="low"), data.sel(field="close"), 14)

    # the relative strength index:
    rsi = qnta.rsi(data.sel(field="close"))

    # the logarithm of the closing price:
    #price = data.sel(field="close").ffill("time").bfill("time").fillna(0)  # fill NaN
    #price = np.log(price)

    # new feature: Trix (TRIX)
    trix = qnta.trix(data.sel(field="close"), 15)  # Using a period of 15 (can be adjusted)

    # new feature: Average True Range (ATR)
    atr = qnta.atr(data.sel(field="high"), data.sel(field="low"), data.sel(field="close"), 14).expand_dims(field=["atr"])

    obv = qnta.obv(data.sel(field="close"), data.sel(field="vol")).expand_dims(field=["obv"])


    # combine the features:
    result = xr.concat(
        [macd2_signal.sel(field="close"), volatility, rsi, trix, atr, obv],
        pd.Index(
            ["macd", "volatility", "rsi", "trix", "atr", "obv"],
            name="field"
        )
    )

    # Expand liquidity to match dimensions of result
    liq_expanded = liq.expand_dims(field=result.field)  # Match the "field" dimension

    # Apply liquidity filter (assets with liq == 0 will be excluded)
    result = result.where(liq_expanded > 0.5, drop=True)


    return result.transpose("time", "field", "asset")

In [ ]:
# displaying the features:
my_features = get_features(stock_data)
display(my_features.sel(field="trix").to_pandas())

asset,NAS:AAL,NAS:AAPL,NAS:ABNB,NAS:ACGL,NAS:ADBE,NAS:ADI,NAS:ADP,NAS:ADSK,NAS:AEP,NAS:AKAM,...,NYS:WMB,NYS:WMT,NYS:WRB,NYS:WST,NYS:WY,NYS:XOM,NYS:XYL,NYS:YUM,NYS:ZBH,NYS:ZTS
time,,,,,,,,,,,,,,,,,,,,,
2005-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-06-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-06-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-06-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-06-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-03,NaN,-0.209801,-0.084468,-0.008608,-0.305682,-0.009653,0.033250,0.034581,0.178915,0.070955,...,0.132998,0.172136,-0.066607,0.173718,0.105555,-0.061001,0.014772,-0.169502,0.107900,-0.022068
2025-02-04,NaN,-0.204178,-0.082885,-0.005053,-0.273177,-0.030822,0.050187,0.055621,0.186830,0.098999,...,0.112359,0.199763,-0.052229,0.161858,0.124738,-0.058539,0.045182,-0.146670,0.108975,-0.002431
2025-02-05,NaN,-0.197666,-0.082791,-0.003332,-0.244006,-0.049473,0.067896,0.076578,0.193517,0.125735,...,0.095841,0.230557,-0.034402,0.148164,0.138780,-0.054112,0.080559,-0.124307,0.106551,0.019539


### **Comprando y vendiendo**.





In [ ]:
def get_target_classes(data):
    """Target classes for predicting if price goes up, down, or stays the same."""

    # Calcular la variación porcentual diaria
    price_change_ratio = qnta.change(data.sel(field="close")) / data.sel(field="close")
    future_price_change_ratio = price_change_ratio.shift(time=-1).fillna(0)  # Manejo de NaNs

    # Definir clases
    class_positive = 1  # Price goes up more than move
    class_neutral = 0   # Price does not move more than move
    class_negative = -1 # Price goes down more than move

    # Umbral de movimiento (ajustar según sea necesario)
    move = 0.02  # 1%

    # Clasificar los rendimientos futuros
    target_price = xr.where(
        future_price_change_ratio < -move, class_negative, future_price_change_ratio
    )
    target_price = xr.where(
        future_price_change_ratio > move, class_positive, target_price
    )
    target_price = xr.where(abs(future_price_change_ratio) <= move, class_neutral, target_price)

    return target_price


### **Comprando**.



In [ ]:
def get_target_classes(data):
    """ Target classes for predicting if price goes up or down."""

    price_current = data.sel(field="close")
    price_future  = qnta.shift(price_current, -1)

    class_positive = 1 # prices goes up
    class_negative = 0 # price goes down

    target_price_up = xr.where(price_future > price_current, class_positive, class_negative)

    return target_price_up

### **Vendiendo**.



In [ ]:
def get_target_classes(data):
    """Target classes for predicting if price goes down or stays the same/up."""

    price_current = data.sel(field="close")
    price_future  = qnta.shift(price_current, -1)

    class_down  = -1  # price goes down
    class_same  = 0   # price stays the same or goes up

    target_price_up = xr.where(price_future < price_current, class_down, class_same)

    return target_price_up


In [ ]:
# displaying the target classes:
my_targetclass = get_target_classes(stock_data)
display(my_targetclass.to_pandas())

asset,NAS:AAL,NAS:AAPL,NAS:ABNB,NAS:ACGL,NAS:ADBE,NAS:ADI,NAS:ADP,NAS:ADSK,NAS:AEP,NAS:AKAM,...,NYS:WMB,NYS:WMT,NYS:WRB,NYS:WST,NYS:WY,NYS:XOM,NYS:XYL,NYS:YUM,NYS:ZBH,NYS:ZTS
time,,,,,,,,,,,,,,,,,,,,,
2005-06-01,0.0,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2005-06-02,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2005-06-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2005-06-06,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2005-06-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-03,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-1.0,1.0,1.0,0.0,0.0,0.0
2025-02-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-02-05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0


In [ ]:
def get_model():
    """Constructor para el modelo ML: HistGradientBoostingClassifier."""
    model = HistGradientBoostingClassifier()
    return model

In [ ]:
# Create and train the models working on an asset-by-asset basis.

asset_name_all = stock_data.coords["asset"].values

target_assets = set(my_targetclass.coords['asset'].values)
feature_assets = set(my_features.coords['asset'].values)
common_assets = target_assets.intersection(feature_assets)

models = dict()

for asset_name in common_assets:
        target_cur = my_targetclass.sel(asset=asset_name).dropna("time", "any")
        features_cur = my_features.sel(asset=asset_name).dropna("time", "any")

        # align features and targets:
        target_for_learn_df, feature_for_learn_df = xr.align(target_cur, features_cur, join="inner")

        if len(features_cur.time) < 10:
            # not enough points for training
                continue
            # HistGradientBoostingClassifier requires targets as 1D arrays
        target_for_learn = target_for_learn_df.values.ravel()
        features_for_learn = feature_for_learn_df.values

        model = get_model()

        try:
            model.fit(feature_for_learn_df.values, target_for_learn_df)
            models[asset_name] = model

        except:
            logging.exception("model training failed")


/tmp/ipykernel_283/2811938951.py:12: FutureWarning:

Passing 'how' as positional argument(s) to dropna was deprecated in version v2023.10.0 and will raise an error two releases later. Please pass them as keyword arguments.

/tmp/ipykernel_283/2811938951.py:13: FutureWarning:

Passing 'how' as positional argument(s) to dropna was deprecated in version v2023.10.0 and will raise an error two releases later. Please pass them as keyword arguments.

/tmp/ipykernel_283/2811938951.py:12: FutureWarning:

Passing 'how' as positional argument(s) to dropna was deprecated in version v2023.10.0 and will raise an error two releases later. Please pass them as keyword arguments.

/tmp/ipykernel_283/2811938951.py:13: FutureWarning:

Passing 'how' as positional argument(s) to dropna was deprecated in version v2023.10.0 and will raise an error two releases later. Please pass them as keyword arguments.

/tmp/ipykernel_283/2811938951.py:12: FutureWarning:

Passing 'how' as positional argument(s) to dropna w

In [ ]:
# Prediction and generating output weights:
weights = xr.zeros_like(stock_data.sel(field="close"))

for asset_name in asset_name_all:
    if asset_name in models:
        model = models[asset_name]
        features_all = my_features
        features_cur = features_all.sel(asset=asset_name).dropna("time", "any")
        if len(features_cur.time) < 1:
            continue
        try:
            # HistGradientBoostingClassifier outputs probabilities, so we take class probabilities for class 1
            probs = model.predict_proba(features_cur.values)[:, 1]
            weights.loc[dict(asset=asset_name, time=features_cur.time.values)] = probs
        except KeyboardInterrupt as e:
            raise e
        except:
            logging.exception("model prediction failed")

print(weights)

/tmp/ipykernel_283/587938141.py:8: FutureWarning:

Passing 'how' as positional argument(s) to dropna was deprecated in version v2023.10.0 and will raise an error two releases later. Please pass them as keyword arguments.

/tmp/ipykernel_283/587938141.py:8: FutureWarning:

Passing 'how' as positional argument(s) to dropna was deprecated in version v2023.10.0 and will raise an error two releases later. Please pass them as keyword arguments.

/tmp/ipykernel_283/587938141.py:8: FutureWarning:

Passing 'how' as positional argument(s) to dropna was deprecated in version v2023.10.0 and will raise an error two releases later. Please pass them as keyword arguments.

/tmp/ipykernel_283/587938141.py:8: FutureWarning:

Passing 'how' as positional argument(s) to dropna was deprecated in version v2023.10.0 and will raise an error two releases later. Please pass them as keyword arguments.

/tmp/ipykernel_283/587938141.py:8: FutureWarning:

Passing 'how' as positional argument(s) to dropna was depreca

<xarray.DataArray 'stocks_s&p500' (time: 4955, asset: 516)> Size: 20MB
array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.70805439, 0.09243003, ..., 0.26291945, 0.35193618,
        0.9782109 ],
       [0.        , 0.54282909, 0.91324693, ..., 0.99622362, 0.96250519,
        0.9574654 ],
       [0.        , 0.65818579, 0.99899853, ..., 0.99538103, 0.95102914,
        0.95915812]])
Coordinates:
  * time     (time) datetime64[ns] 40kB 2005-06-01 2005-06-02 ... 2025-02-07
    field    <U5 20B 'close'
  * asset    (asset) <U9 19kB 'NAS:AAL' 'NAS:AAPL' ... 'NYS:ZBH' 'NYS:ZTS'


/tmp/ipykernel_283/587938141.py:8: FutureWarning:

Passing 'how' as positional argument(s) to dropna was deprecated in version v2023.10.0 and will raise an error two releases later. Please pass them as keyword arguments.



In [ ]:
def get_sharpe(stock_data, weights):
    """Calculates the Sharpe ratio"""
    rr = qnstats.calc_relative_return(stock_data, weights)
    sharpe = qnstats.calc_sharpe_ratio_annualized(rr).values[-1]
    return sharpe

sharpe = get_sharpe(stock_data, weights)
sharpe

0.13589992660360797

The sharpe ratio using the method above follows from **forward looking**. Predictions for (let us say) 2017 know about the relation between features and targets in 2020. Let us visualize the results:

In [ ]:
import qnt.graph as qngraph

statistics = qnstats.calc_stat(stock_data, weights)

display(statistics.to_pandas().tail())

performance = statistics.to_pandas()["equity"]
qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

display(statistics[-1:].sel(field = ["sharpe_ratio"]).transpose().to_pandas())

# check for correlations with existing strategies:
qnstats.print_correlation(weights,stock_data)

field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2025-02-03,1.528732,-0.005116,0.160301,-0.042810,-0.627265,0.136235,0.021838,1.0,515.0,0.229848,8.325759
2025-02-04,1.528877,0.000095,0.160284,-0.042719,-0.627265,0.136251,0.021839,1.0,515.0,0.229849,8.325574
2025-02-05,1.535945,0.004623,0.160271,-0.038293,-0.627265,0.137731,0.022074,1.0,515.0,0.229840,8.325415
2025-02-06,1.534878,-0.000695,0.160255,-0.038961,-0.627265,0.137491,0.022034,1.0,515.0,0.229836,8.325351
2025-02-07,1.527449,-0.004840,0.160243,-0.043613,-0.627265,0.135900,0.021777,1.0,515.0,0.229834,8.325398


time,2025-02-07
field,
sharpe_ratio,0.1359


WARNING! This strategy correlates with other strategies and will be rejected.
Modify the strategy to produce the different output.


The number of systems with a larger Sharpe ratio and correlation larger than 0.9: 45
The max correlation value (with systems with a larger Sharpe ratio): 0.9933256208838918
Current sharpe ratio(3y): 0.10025403246904133

Correlated examples:

Name                                                  Coefficient    Sharpe ratio
--------------------------------------------------  -------------  --------------
Q18 Machine Learning on a Rolling Basis - disabled       0.907995        0.408382


Let us now use the Quantiacs **backtester** for avoiding **forward looking**.

The backtester performs some transformations: it trains the model on one slice of data (using only data from the past) and predicts the weights for the following slice on a rolling basis:

In [ ]:
"""R2 (coefficient of determination) regression score function."""
r2_score(my_targetclass, weights, multioutput="variance_weighted")

-3.148248856227359

In [ ]:
"""The explained variance score explains the dispersion of errors of a given dataset"""
explained_variance_score(my_targetclass, weights, multioutput="uniform_average")

-0.6663409233298525

In [ ]:
"""The explained variance score explains the dispersion of errors of a given dataset"""
mean_absolute_error(my_targetclass, weights)

0.8168362820336263

Let us now use the Quantiacs **backtester** for avoiding **forward looking**.

The backtester performs some transformations: it trains the model on one slice of data (using only data from the past) and predicts the weights for the following slice on a rolling basis:

In [ ]:
def train_model(data):
    """Create and train the model working on an asset-by-asset basis."""

    asset_name_all = data.coords["asset"].values
    target_all = get_target_classes(data)
    features_all = get_features(data)

    # Alinear ambos conjuntos de datos para garantizar consistencia
    target_all, features_all = xr.align(target_all, features_all, join="inner")

    models = dict()

    for asset_name in features_all.coords["asset"].values:
        if asset_name not in target_all.coords["asset"].values:
            continue  # Omitir activos que no están presentes en ambos conjuntos
        # Drop missing values:
        target_cur = target_all.sel(asset=asset_name).dropna("time", "any")
        features_cur = features_all.sel(asset=asset_name).dropna("time", "any")

        # Align features and targets:
        target_for_learn_df, feature_for_learn_df = xr.align(target_cur, features_cur, join="inner")

        # Verificar si hay suficientes datos
        if len(features_cur.time) < 10:
            continue

        # Convertir a arrays planos (1D para targets y 2D para features)
        target_for_learn = target_for_learn_df.values.ravel()
        features_for_learn = feature_for_learn_df.values

        model = get_model()

        try:
            model.fit(features_for_learn, target_for_learn)
            models[asset_name] = model

        except Exception:
            logging.exception("model training failed")

    return models


In [ ]:
def predict_weights(models, data):
    """The model predicts if the price is going up or down.
       The prediction is performed for several days in order to speed up the evaluation."""

    asset_name_all = data.coords["asset"].values
    weights = xr.zeros_like(data.sel(field="close"))

    for asset_name in asset_name_all:
        if asset_name in models:
            model = models[asset_name]
            features_all = get_features(data)
            features_cur = features_all.sel(asset=asset_name).dropna("time", "any")

            if len(features_cur.time) < 10:
                continue

            try:
                # Obtener probabilidades para la clase 1
                probs = model.predict_proba(features_cur.values)[:, 1]
                weights.loc[dict(asset=asset_name, time=features_cur.time.values)] = probs

            except KeyboardInterrupt as e:
                raise e

            except Exception:
                logging.exception("model prediction failed")

    return weights


In [ ]:
# Calculate weights using the backtester:
weights = qnbt.backtest_ml(
    train                         = train_model,
    predict                       = predict_weights,
    train_period                  =  2 *365,  # the data length for training in calendar days
    retrain_interval              = 10 *365,  # how often we have to retrain models (calendar days)
    retrain_interval_after_submit = 1,        # how often retrain models after submission during evaluation (calendar days)
    predict_each_day              = False,    # Is it necessary to call prediction for every day during backtesting?
                                              # Set it to True if you suspect that get_features is looking forward.
    competition_type              = "stocks_s&p500",  # competition type
    lookback_period               = 365,                 # how many calendar days are needed by the predict function to generate the output
    start_date                    = "2005-01-01",        # backtest start date
    analyze                       = True,
    build_plots                   = True  # do you need the chart?
)

Run the last iteration...


100% (79197 of 79197) |##################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (13101656 of 13101656) |############| Elapsed Time: 0:00:00 Time:  0:00:00


fetched chunk 1/2 6s


100% (3389068 of 3389068) |##############| Elapsed Time: 0:00:00 Time:  0:00:00


fetched chunk 2/2 8s
Data loaded 8s


...

fetched chunk 13/13 10s
Data loaded 10s
Output cleaning...
fix uniq
ffill if the current price is None...
Check liquidity...
Ok.
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.
Write output: /root/fractions.nc.gz
State saved.
---
Analyze results...
Check...
Check liquidity...
Ok.
Check missed dates...
Ok.
Check the sharpe ratio...


WARNING! There are not enough points in the output.
The output series should start from 2006-01-01 or earlier instead of 2015-01-02


Period: 2006-01-01 - 2025-02-07
Sharpe Ratio = 0.48565423775975447


ERROR! The Sharpe Ratio is too low. 0.48565423775975447 < 0.7
Improve the strategy and make sure that the in-sample Sharpe Ratio more than 0.7.


---
Align...
Calc global stats...
---
Calc stats per asset...


The Sharpe ratio is obviously smaller as the training process is not looking forward (as it happens by processing data on a global basis), but performed on a rolling basis.